# Load Data (already cleaned)

In [80]:
# Manage imports
import numpy as np
import pandas as pd
import seaborn as sns  
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'

In [116]:
# read data in memory
train = pd.read_csv("C:\\Users\\Leo\\TaxiData\\clean_v2.csv",index_col=0)

E:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
train.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,...,mta_tax,tip_amount,tolls_amount,total_amount,duration,duration_seconds,pickupDay,pickupHour,ridesAtPickupHour,avg_speed
0,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,-73.977463,40.753979,2,...,0.5,0.00,0.0,7.30,0 days 00:06:32.000000000,392,9,21,21285,7.255102
1,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,-73.981636,40.670242,1,...,0.5,4.00,0.0,27.30,0 days 00:28:35.000000000,1715,9,21,21285,10.957434
3,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,-73.929466,40.851540,1,...,0.5,1.00,0.0,28.30,0 days 00:29:34.000000000,1774,9,21,21285,14.996618
4,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,-73.985909,40.766445,1,...,0.5,2.96,0.0,17.76,0 days 00:16:47.000000000,1007,9,21,21285,11.082423
5,2016-06-09 21:06:36,2016-06-09 21:19:21,1,2.17,-73.995201,40.739491,1,-73.993202,40.762642,1,...,0.5,2.36,0.0,14.16,0 days 00:12:45.000000000,765,9,21,21285,10.211765


In [19]:
max(test.tpep_pickup_datetime)

'2016-05-31 23:59:36'

In [126]:
test = pd.read_csv("C:\\Users\\Leo\\Dropbox\\Info Göttingen\\Practical Course Data Science\\Task 2\\data\\test_data_v2.csv",engine="python")

# Prepare the Test set (add features)

## Traffic at pick up time (has to be changed?)

In [20]:
traffic = train.groupby(["pickupDay","pickupHour"])

In [21]:
traffic

In [ ]:
%%time
test["ridesAtPickupHour"] = test.apply(lambda x: traffic.trip_distance.get_group((x.pickupDay,x.pickupHour)).shape[0],axis=1 )

Problem: does not make sense, should be done per weekday average?

## Add duration in seconds

In [162]:
test["tpep_pickup_datetime"] = pd.to_datetime(test.tpep_pickup_datetime)
test['tpep_dropoff_datetime'] = pd.to_datetime(test.tpep_dropoff_datetime)

In [163]:
# Get duration
test['duration'] = pd.to_datetime(test["tpep_dropoff_datetime"])-pd.to_datetime(test["tpep_pickup_datetime"])

In [164]:
%%time
test["duration_seconds"] = test.apply(lambda x : x.duration.total_seconds(),axis=1)

Wall time: 3.62 s


## Calculate average speed

In [165]:
test = test.assign(avg_speed = test.trip_distance/test.duration_seconds*60*60)

In [166]:
test.avg_speed.describe()

count    64000.000000
mean        14.546904
std          9.388043
min          0.014337
25%          8.153310
50%         11.716077
75%         18.140034
max        176.170213
Name: avg_speed, dtype: float64

# Ratecode 1 (Standard rate)

## Building the Model

In [12]:
code1data = train[train.RatecodeID == 1]

In [37]:
code1test = test[test.RatecodeID == 1]

In [45]:
import lightgbm as lgb

features = ["duration_seconds","passenger_count","trip_distance","payment_type","avg_speed"]

code1lgb_dataset = lgb.Dataset(code1data[features], label=code1data["fare_amount"])

params = {}
params['learning_rate'] = .05
params['boosting_type'] = 'gbdt'
params['min_samples_split'] = 3
params['min_samples_leaf'] = 15
params['max_features'] = 'sqrt'
params['max_depth'] = 10
params['subsample'] = 0.8

code1_model = lgb.train(params, code1lgb_dataset, 300)


## Prediction

In [38]:
code1prediction = np.round(code1_model.predict(code1test[features])*2)/2

In [39]:
code1prediction

array([ 4. ,  6.5, 10.5, ...,  4.5,  6. , 15. ])

In [40]:
code1test = code1test.assign(fare_amount_prediction = code1prediction)

# Ratecode 2 (JFK)

## Building the Model

No model building neccessary, all rides cost 52$

In [118]:
code2data = train[train.RatecodeID == 2]
code2test = test[test.RatecodeID == 2]

## Prediction

In [119]:
code2test["fare_amount_prediction"] = 52

# Ratecode 3 (Newark)

## Building the Model

In [44]:
code3data = train[train.RatecodeID == 3]

In [46]:
code3test = test[test.RatecodeID == 3]

In [59]:
features = ["duration_seconds","passenger_count","trip_distance","payment_type","avg_speed"]

code3lgb_dataset = lgb.Dataset(code3data[features], label=code3data["fare_amount"])

params = {}
params['learning_rate'] = .05
params['boosting_type'] = 'gbdt'
params['min_samples_split'] = 3
params['min_samples_leaf'] = 15
params['max_features'] = 'sqrt'
params['max_depth'] = 10
params['subsample'] = 0.8

code3_model = lgb.train(params, code3lgb_dataset, 300)

## Prediction

In [60]:
code3prediction = np.round(code3_model.predict(code3test[features])*2)/2
code3test = code3test.assign(fare_amount_prediction = code3prediction)

# Ratecode 4 (Nassau or Westchester)

## Building the Model

In [167]:
code4data = train[train.RatecodeID == 4]
code4test = test[test.RatecodeID == 4]
np.shape(code4test)

(1000, 18)

In [168]:
#shapely can be installed via anaconda -> environments
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [169]:
# returns whether the point given is in the area given
def inArea(polygon,lat,long):
    point = Point(lat,long);
    poly = Polygon(polygon)
    return poly.contains(point)    

In [170]:
#cords for a polygon around manhatten, manually set via google maps
manhattan_cords = [(40.882207, -73.933869), #1
         (40.872343, -73.908292), #2
         (40.836117, -73.933011), #3
         (40.804680, -73.931123), #4
         (40.798833, -73.918591), #5
         (40.739289, -73.967000), #6
         (40.707155, -73.973866), #7
         (40.699217, -74.021416), #8
         (40.759316, -74.012662)] #9

In [171]:
#apply to train set
code4data["endsInManhattan"] = code4data.apply(lambda x : inArea(manhattan_cords,x["dropoff_latitude"],x["dropoff_longitude"]),axis=1)
code4data["startsInManhattan"] = code4data.apply(lambda x : inArea(manhattan_cords,x["pickup_latitude"],x["pickup_longitude"]),axis=1)

In [172]:
from geopy.distance import geodesic
from geopy.distance import great_circle

In [173]:
# all bridges and tunnels to/from manhattan, bridges marked with ! are highways or tunnels and more likely to be used by taxis
bridges = [ (40.877720, -73.922362), # 0 henry hudson bridge !
          (40.873648, -73.911075), # 1 broadway !
          (40.862840, -73.914986), # 2 university heights bridge
          (40.846674, -73.927806), # 3 washington bridge - close to v
          (40.845522, -73.928256), # 4 alexander hamilton bridge !
          (40.828062, -73.933873), # 5 macombs dam bridge
          (40.819489, -73.933058), # 6 145th st bridge - close to v
          (40.814081, -73.933187), # 7 madison avenue bridge
          (40.807591, -73.932399), # 8 third avenue bridge
           (40.803516, -73.928818),# 9 willis ave bridge
           (40.800368, -73.927835), # 10 robert f kennedy bridge !
           (40.756681, -73.954185), # 11 ed koch queensboro bridge !
           (40.745106, -73.964063), # 12 queens midtown tunnel !
           (40.713407, -73.972299), # 13 williamsburg bridge
           (40.707095, -73.990500), # 14 manhattan bridge
           (40.705972, -73.996851), # 15 brooklyn bridge !
           (40.701161, -74.015692), # 16 Hugh l Carey Tunnel
           (40.727487, -74.020880), # 17 holland tunnel
           (40.763370, -74.010028), # 18 lincoln tunnel !
           (40.851644, -73.952118), # 19 george washington bridge !
          ]

In [174]:
# try to find out which bridge is used in a trip 
# assumption: the bridge is used that minimizes the pickup-bridge-dropoff distance (air-line distance)
counter = 1
for index, row in code4data.iterrows():
    # 4 options:
    # starts in city, ends in city --> should not have ratecode 4 -> outlier, ignore
    # starts in city, ends out of city -> standard trip
    # starts out of city, ends out of city -> no calculations to be done here, use double rate for whole trip
    # starts out of city, ends in city -> reverse standard trip
    startP = (row["pickup_latitude"],row["pickup_longitude"])
    endP = (row["dropoff_latitude"],row["dropoff_longitude"])
    
    if(row.startsInManhattan and row.endsInManhattan):
        #print("Outlier found, index",index)
        continue
    elif (not row.startsInManhattan and not row.endsInManhattan):
        #print("Complete out of city trip, index",index)
        code4data.at[index,"Bridge"] = None;
        code4data.at[index,"in_city_distance"] = 0;
        code4data.at[index,"out_of_city_distance"] = geodesic(startP, endP).miles
        code4data.at[index,"total_over_bridge_distance"] = geodesic(startP, endP).miles  
        continue
    elif (not row.startsInManhattan and  row.endsInManhattan): # reverse trip
        temp = startP # here we swap start and endpoints so its similar to a standard trip
        startP = endP
        endP = temp
    
    #find closest bridge
    closest_bridge_index = -1
    closest_total_bridge_distance = float('inf')
    for i in range(len(bridges)):
        distance_to_bridge = geodesic(startP, bridges[i]).miles
        distance_to_dest =  geodesic(bridges[i],endP).miles
        total = distance_to_bridge + distance_to_dest
        if(total < closest_total_bridge_distance):
            closest_total_bridge_distance = total
            closest_bridge_index = i
            closest_distance_to_bridge = distance_to_bridge
            closest_distance_to_dest = distance_to_dest
            
    #print("Route {} travels over bridge/tunnel {} (to bridge: {:0.2f}, to dest: {:0.2f}, total: {:0.2f}, trip_distance : {:0.2f})".format(
    #    counter, closest_bridge_index,closest_distance_to_bridge,closest_distance_to_dest,closest_total_bridge_distance,totalDistance))
    counter = counter + 1
    
    code4data.at[index,"Bridge"] = closest_bridge_index;
    code4data.at[index,"in_city_distance"] = closest_distance_to_bridge;
    code4data.at[index,"out_of_city_distance"] = closest_distance_to_dest
    code4data.at[index,"total_over_bridge_distance"] = closest_total_bridge_distance       

In [175]:
#states how much of the fare was contributed by ride in the city. Calculated by 
code4data["inCityPercentage"] = code4data.apply(lambda x: x.in_city_distance/(x.in_city_distance+2*x.out_of_city_distance+0.0001),axis=1)

In [176]:
#2.5 were subtracted from fare amount (inital charge)
code4data["inCityFare"] = code4data.apply(lambda x: x.inCityPercentage*(x.fare_amount-2.5),axis=1)
code4data["outCityFare"] = code4data.apply(lambda x: (1-x.inCityPercentage)*(x.fare_amount-2.5),axis=1)

Now we have to train three models. Two for trips that start in the city and end out of it and vice versa, thus using both the standard and the doubled rate (one model for each). The predctions are then add up. The third model is for trips that are completly out of the city and therefore only use the doubled rate. Here more information can be used from the dataset, because we do not have to make assumptions on where the trip was split.

In [177]:
code4data_mixed = code4data[code4data.Bridge.notnull()] # trips that used a bridge -> entered/left manhattan
np.shape(code4data_mixed)

(777, 31)

In [178]:
code4data_out = code4data[-code4data.Bridge.notnull()] # trips that not used a bridge -> never were in Manhattan
np.shape(code4data_out)

(3429, 31)

In [179]:
#train models for the in city and out of city parts
code4_model_inner = RandomForestRegressor(random_state=1337)
code4_model_outer = RandomForestRegressor(random_state=1338)

#chose features
mixed_inner_train = code4data_mixed[["passenger_count","in_city_distance","payment_type"]]
mixed_outer_train = code4data_mixed[["passenger_count", "out_of_city_distance","payment_type"]]

#fit the models
code4_model_inner.fit(mixed_inner_train,code4data_mixed["inCityFare"])
code4_model_outer.fit(mixed_outer_train,code4data_mixed["outCityFare"]);

Next: model for outer trips.


In [180]:
code4_model_out = RandomForestRegressor(random_state=1339)
# make train set
features = ["passenger_count","trip_distance","payment_type","duration_seconds"]

#fit - subtract 2.5 for inital charge, has to be added later
code4_model_out.fit(code4data_out[features],code4data_out["fare_amount"]-2.5)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1339, verbose=0, warm_start=False)

## Prediction

In [181]:
#apply to test set
code4test["endsInManhattan"] = code4test.apply(lambda x : inArea(manhattan_cords,x["dropoff_latitude"],x["dropoff_longitude"]),axis=1)
code4test["startsInManhattan"] = code4test.apply(lambda x : inArea(manhattan_cords,x["pickup_latitude"],x["pickup_longitude"]),axis=1)

We have four possible trip patterns:
    1. starts in city, ends in city --> should not have ratecode 4 -> outlier, ignore
    2. starts in city, ends out of city -> standard trip
    3. starts out of city, ends out of city -> no calculations to be done here, use double rate for whole trip
    4. starts out of city, ends in city -> reverse standard trip

In [182]:
np.shape(code4test)

(1000, 20)

In [183]:
#option1: how many trips start and end in manhattan, these should not have this ratecode
code4test_wrong = code4test[(code4test.startsInManhattan & code4test.endsInManhattan)]
np.shape(code4test_wrong)

(35, 20)

In [184]:
#option2 & option 4: 
code4test_mixed = code4test[code4test.startsInManhattan ^ code4test.endsInManhattan] 
np.shape(code4test_mixed)

(238, 20)

In [185]:
#option 3:
code4test_out = code4test[(code4test.startsInManhattan==False) & (code4test.endsInManhattan==False)] 
np.shape(code4test_out)

(727, 20)

In [186]:
# try to find out which bridge is used in a trip 
# assumption: the bridge is used that minimizes the pickup-bridge-dropoff distance (air-line distance)
counter = 1
for index, row in code4test_mixed.iterrows():
    
    startP = (row["pickup_latitude"],row["pickup_longitude"])
    endP = (row["dropoff_latitude"],row["dropoff_longitude"])
    
    if(row.startsInManhattan and row.endsInManhattan):
        #print("Outlier found, index",index)
        continue
    elif (not row.startsInManhattan and not row.endsInManhattan):
        #print("Complete out of city trip, index",index)
        code4test_mixed.at[index,"bridge"] = None;
        code4test_mixed.at[index,"in_city_distance"] = 0;
        code4test_mixed.at[index,"out_of_city_distance"] = geodesic(startP, endP).miles
        code4test_mixed.at[index,"total_over_bridge_distance"] = geodesic(startP, endP).miles  
        continue
    elif (not row.startsInManhattan and  row.endsInManhattan): # reverse trip
        temp = startP # here we swap start and endpoints so its similar to a standard trip
        startP = endP
        endP = temp
    
    #find closest bridge
    closest_bridge_index = -1
    closest_total_bridge_distance = float('inf')
    for i in range(len(bridges)):
        distance_to_bridge = geodesic(startP, bridges[i]).miles
        distance_to_dest =  geodesic(bridges[i],endP).miles
        total = distance_to_bridge + distance_to_dest
        if(total < closest_total_bridge_distance):
            closest_total_bridge_distance = total
            closest_bridge_index = i
            closest_distance_to_bridge = distance_to_bridge
            closest_distance_to_dest = distance_to_dest
            
    #print("Route {} travels over bridge/tunnel {} (to bridge: {:0.2f}, to dest: {:0.2f}, total: {:0.2f}, trip_distance : {:0.2f})".format(
    #    counter, closest_bridge_index,closest_distance_to_bridge,closest_distance_to_dest,closest_total_bridge_distance,totalDistance))
    counter = counter + 1
    
    code4test_mixed.at[index,"bridge"] = closest_bridge_index;
    code4test_mixed.at[index,"in_city_distance"] = closest_distance_to_bridge;
    code4test_mixed.at[index,"out_of_city_distance"] = closest_distance_to_dest
    code4test_mixed.at[index,"total_over_bridge_distance"] = closest_total_bridge_distance       

In [190]:
#states how much of the fare was contributed by ride in the city. Calculated by 
code4test_mixed["inCityPercentage"] = code4test_mixed.apply(lambda x: x.in_city_distance/(x.in_city_distance+2*x.out_of_city_distance+0.0001),axis=1)

In [191]:
# prediction for the inner city part
code4_mixed_inner_pred = code4_model_inner.predict(code4test_mixed[["passenger_count","in_city_distance","payment_type"]])

In [192]:
#prediction for out of city part
code4_mixed_outer_pred = code4_model_outer.predict(code4test_mixed[["passenger_count","out_of_city_distance","payment_type"]])

In [193]:
#combine the results
#round comined predictions to closest .5
#add 2.5 inital charge which was removed before training
code4test_mixed["fare_amount_prediction"]=np.round((code4_mixed_inner_pred+code4_mixed_outer_pred)*2)/2+2.5

Next: predict out trips.

In [194]:
code4test_out_pred = code4_model_out.predict(code4test_out[["passenger_count","trip_distance","payment_type","duration_seconds"]])
code4test_out["fare_amount_prediction"]=np.round(code4test_out_pred*2)/2+2.5

Lastly: predict trips that only were in Manhattan with double rate model (same as outer trips)

In [195]:
code4_test_wrong_pred = code4_model_out.predict(code4test_wrong[["passenger_count","trip_distance","payment_type","duration_seconds"]])
code4test_wrong["fare_amount_prediction"]=np.round(code4_test_wrong_pred*2)/2+2.5

In [196]:
#combine all code 4 dataframes with predictions
code4test = pd.concat([code4test_mixed,code4test_out,code4test_wrong])

# Ratecode 5 (Negotiated Fares)

## Building the Model

In [111]:
code5data = train[train.RatecodeID == 5]
code5test = test[test.RatecodeID == 5]

In [112]:
features = ["duration_seconds","passenger_count","trip_distance","payment_type","avg_speed"]

code5lgb_dataset = lgb.Dataset(code5data[features], label=code5data["fare_amount"])

params = {}
params['learning_rate'] = .05
params['boosting_type'] = 'gbdt'
params['min_samples_split'] = 3
params['min_samples_leaf'] = 15
params['max_features'] = 'sqrt'
params['max_depth'] = 10
params['subsample'] = 0.8

code5_model = lgb.train(params, code5lgb_dataset, 300)

## Prediction

In [114]:
code5prediction = np.round(code5_model.predict(code5test[features])*2)/2
code5test = code5test.assign(fare_amount_prediction = code5prediction)

# Recombine all predictions and save

In [197]:
testResult = pd.concat([code1test,code2test,code3test,code4test,code5test])

In [198]:
testResult.to_csv("C:\\Users\\Leo\\Dropbox\\Info Göttingen\\Practical Course Data Science\\Task 2\\data\\test_prediction.csv")

In [199]:
testResult.fare_amount_prediction.describe()

count    64000.000000
mean        23.213414
std         23.945688
min          3.500000
25%          7.500000
50%         12.000000
75%         31.500000
max        195.500000
Name: fare_amount_prediction, dtype: float64

In [200]:
test.describe()

,id,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,tolls_amount,improvement_surcharge,duration,duration_seconds,avg_speed
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.00000,64000.000000,64000,64000.000000,64000.000000
mean,32000.500000,1.537047,1.671188,5.896713,-73.960769,40.744481,1.406250,-73.975568,40.746855,1.337031,1.74787,0.299991,0 days 00:20:30.515343,1230.515344,14.546904
std,18475.352951,0.498630,1.303686,7.234262,0.060701,0.037676,0.913769,0.084823,0.047903,0.492926,4.14628,0.001677,0 days 00:18:00.955691,1080.955692,9.388043
min,1.000000,1.000000,0.000000,0.010000,-74.527763,40.535580,1.000000,-74.833679,40.045311,1.000000,0.00000,0.000000,0 days 00:00:40,40.000000,0.014337
25%,16000.750000,1.000000,1.000000,1.200000,-73.991264,40.732536,1.000000,-73.993828,40.725780,1.000000,0.00000,0.300000,0 days 00:07:48,468.000000,8.153310
50%,32000.500000,2.000000,1.000000,2.260000,-73.980370,40.752197,1.000000,-73.980309,40.751484,1.000000,0.00000,0.300000,0 days 00:14:18,858.000000,11.716077
75%,48000.250000,2.000000,2.000000,8.080000,-73.962111,40.766815,1.000000,-73.960541,40.768697,2.000000,0.00000,0.300000,0 days 00:27:39,1659.000000,18.140034
max,64000.000000,2.000000,9.000000,86.600000,-72.659561,41.255787,5.000000,-72.551582,41.525589,4.000000,82.50000,0.300000,0 days 02:45:31,9931.000000,176.170213
